In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import copy
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction


Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [5]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
#paths = [os.path.join(data_dir, filename) for filename in files]
unloadable_models = ["BIOMD0000000075_url.xml",
                     "BIOMD0000000081_url.xml",
                     "BIOMD0000000094_url.xml",
                     "BIOMD0000000353_url.xml",
                     "BIOMD0000000596_url.xml"]
net_files = list(set(files) - set(unloadable_models))
print(len(net_files))

651


In [6]:
# model statistics columns
MODEL_ID = "model_id"
NUM_TOTAL_REACTIONS = "num_total_reactions"
NUM_BDRY_REACTIONS = "num_bdry_reactions"
NUM_UNIUNI_REACTIONS = "num_uniuni_reactions"
NUM_UMMU_REACTIONS = "num_ummu_reactions"
NUM_MULTIMULTI_REACTIONS = "num_multimulti_reactions"
#

In [ ]:
results = pd.DataFrame(0, index=files, columns=result_columns)
results[:5]

In [26]:
num_uniuni = 0
num_ummu = 0
num_dic = {cn.REACTION_1_1: 0,
                 cn.REACTION_1_n: 0,
                 cn.REACTION_n_1: 0
                }

In [27]:
for reaction in simple.reactions:
  for category in num_dic.keys():
    if reaction.category == category:
      print("reaction", reaction.label, "is ", category)
      print("num_dic, category is", num_dic[category])
      num_dic[category] += 1
num_dic

reaction ATPase is  reaction_1_1
num_dic, category is 0
reaction ENO is  reaction_1_1
num_dic, category is 1
reaction FBA is  reaction_1_n
num_dic, category is 0
reaction GPM is  reaction_1_1
num_dic, category is 2
reaction GPP is  reaction_1_1
num_dic, category is 3
reaction PDC is  reaction_1_1
num_dic, category is 4
reaction PGI is  reaction_1_1
num_dic, category is 5
reaction PGM is  reaction_1_1
num_dic, category is 6
reaction TPI is  reaction_1_1
num_dic, category is 7
reaction TPP is  reaction_1_1
num_dic, category is 8
reaction UGP is  reaction_n_1
num_dic, category is 0
reaction HXT is  reaction_1_1
num_dic, category is 9
reaction RKI is  reaction_1_1
num_dic, category is 10
reaction RPE is  reaction_1_1
num_dic, category is 11
reaction SOL is  reaction_1_1
num_dic, category is 12
reaction NADPH_oxidase is  reaction_1_1
num_dic, category is 13


{'reaction_1_1': 14, 'reaction_1_n': 1, 'reaction_n_1': 1}

In [13]:
# Checking all graphs 
count = 0
simple = SimpleSBML()
res_list = []
false_errors = set()
for file in net_files:
  num_uniuni = 0
  num_ummu = 0
  num_multimulti = 0
  num_bdry = 0
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  simple.initialize(os.path.join(data_dir, file))
  num_total_reactions = len(simple.reactions)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600


In [31]:
# Checking all graphs 
count = 0
simple = SimpleSBML()
res_list = []
false_errors = set()
for file in net_files:
  num_uniuni = 0
  num_ummu = 0
  num_multimulti = 0
  num_bdry = 0
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  simple.initialize(os.path.join(data_dir, file))
  num_dic = {cn.REACTION_1_1: 0,
             cn.REACTION_1_n: 0,
             cn.REACTION_n_1: 0,
             cn.REACTION_n_n: 0,
             cn.REACTION_BOUNDARY: 0
            }  
  for reaction in simple.reactions:
    for category in num_dic.keys():
      if reaction.category == category:
        num_dic[category] += 1
  res_dic = dict({MODEL_ID: file[:-8],
                  NUM_TOTAL_REACTIONS: len(simple.reactions),
                  NUM_BDRY_REACTIONS: num_dic[cn.REACTION_BOUNDARY],
                  NUM_UNIUNI_REACTIONS: num_dic[cn.REACTION_1_1],
                  NUM_UMMU_REACTIONS: num_dic[cn.REACTION_1_n] + num_dic[cn.REACTION_n_1],
                  NUM_MULTIMULTI_REACTIONS: num_dic[cn.REACTION_n_n]})
  sum = res_dic[NUM_BDRY_REACTIONS] + res_dic[NUM_UNIUNI_REACTIONS] + res_dic[NUM_UMMU_REACTIONS] + res_dic[NUM_MULTIMULTI_REACTIONS]
  if sum != res_dic[NUM_TOTAL_REACTIONS]:
    print("Error! The sum doens't match with model %s" % file)
  res_list.append(res_dic)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600


In [32]:
res_df = pd.DataFrame(res_list)
res_df.head()

,model_id,num_bdry_reactions,num_multimulti_reactions,num_total_reactions,num_ummu_reactions,num_uniuni_reactions
0,BIOMD0000000485,0,0,4,0,4
1,BIOMD0000000153,0,0,152,146,6
2,BIOMD0000000359,0,1,8,5,2
3,BIOMD0000000076,0,0,2,0,2
4,BIOMD0000000392,1,8,27,14,4


In [47]:
print("average total reactions", np.mean(res_df[NUM_TOTAL_REACTIONS]))
print("average bdry reactions", np.mean(res_df[NUM_BDRY_REACTIONS]))
print("Average number of uniuni", np.mean(res_df[NUM_UNIUNI_REACTIONS]))
print("Average fraction of uni uni reactions",
      np.mean(res_df[NUM_UNIUNI_REACTIONS] / res_df[NUM_TOTAL_REACTIONS]))

average total reactions 36.69278033794163
average bdry reactions 5.7910906298003075
Average number of uniuni 12.109062980030721
Average fraction of uni uni reactions 0.36403940743727137


In [57]:
print("se Number total reactions", np.std(res_df[NUM_TOTAL_REACTIONS]) / np.sqrt(len(res_df)))
print("se Number bdry reactions", np.std(res_df[NUM_BDRY_REACTIONS]) / np.sqrt(len(res_df)))
print("se Number of uniuni",
      np.std(res_df[NUM_UNIUNI_REACTIONS]) / np.sqrt(len(res_df)))
print("se Fraction, uni uni reactions",
      np.std(res_df[NUM_UNIUNI_REACTIONS] / res_df[NUM_TOTAL_REACTIONS]) / np.sqrt(len(res_df)))

se Number total reactions 3.6081273739288404
se Number bdry reactions 0.4483080965923733
se Number of uniuni 1.810567518477487
se Fraction, uni uni reactions 0.012625484543655559


In [37]:
print("Average number of UMMU",
     np.mean(res_df[NUM_UMMU_REACTIONS]))
print("Average fraction of UMMU",
     np.mean(res_df[NUM_UMMU_REACTIONS]/res_df[NUM_TOTAL_REACTIONS]))

Average number of UMMU 13.84331797235023
Average fraction of UMMU 0.2473310978579935


In [59]:
print("se number of UMMU",
     np.std(res_df[NUM_UMMU_REACTIONS]) / np.sqrt(len(res_df)))
print("se fraction of UMMU",
     np.std(res_df[NUM_UMMU_REACTIONS]/res_df[NUM_TOTAL_REACTIONS])  / np.sqrt(len(res_df)) )

se number of UMMU 2.1399575261708894
se fraction of UMMU 0.011693589931140719


In [38]:
print("Average number of multi-multi",
     np.mean(res_df[NUM_MULTIMULTI_REACTIONS]))
print("Average fraction of multi-multi",
     np.mean(res_df[NUM_MULTIMULTI_REACTIONS]/res_df[NUM_TOTAL_REACTIONS]))

Average number of multi-multi 4.949308755760368
Average fraction of multi-multi 0.09515350455497812


In [60]:
print("se number of multi-multi",
     np.std(res_df[NUM_MULTIMULTI_REACTIONS]) / np.sqrt(len(res_df)) )
print("se fraction of multi-multi",
     np.std(res_df[NUM_MULTIMULTI_REACTIONS]/res_df[NUM_TOTAL_REACTIONS]) / np.sqrt(len(res_df)))

se number of multi-multi 0.9786276392683663
se fraction of multi-multi 0.007434063376789388


In [46]:
print("Average number of bdry",
     np.mean(res_df[NUM_BDRY_REACTIONS]))
print("Average fraction of bdry",
     np.mean(res_df[NUM_BDRY_REACTIONS]/res_df[NUM_TOTAL_REACTIONS]))

Average number of bdry 5.7910906298003075
Average fraction of bdry 0.2934759901497571


In [61]:
print("se of bdry",
     np.std(res_df[NUM_BDRY_REACTIONS]) / np.sqrt(len(res_df)))
print("se of bdry",
     np.std(res_df[NUM_BDRY_REACTIONS]/res_df[NUM_TOTAL_REACTIONS]) / np.sqrt(len(res_df)))

se of bdry 0.4483080965923733
se of bdry 0.01420549716980877


In [42]:
print("Number of models with no boundary reactions", len(res_df[res_df[NUM_BDRY_REACTIONS]==0]))

Number of models with no boundary reactions 346


In [43]:
print("Number of models with no uniuni reactions", len(res_df[res_df[NUM_UNIUNI_REACTIONS]==0]))

Number of models with no uniuni reactions 194


In [44]:
print("Number of models with no multimulti reactions", len(res_df[res_df[NUM_MULTIMULTI_REACTIONS]==0]))

Number of models with no multimulti reactions 492


In [45]:
print("Number of models with no UMMU reactions", len(res_df[res_df[NUM_UMMU_REACTIONS]==0]))

Number of models with no UMMU reactions 294
